In [1]:
import torch
import numpy as np

In [6]:
num_samples=1000
num_input=2
true_w=[4,-3.5,2]
true_b=[6.1]
features=torch.from_numpy(np.random.normal(0,1,(num_samples,num_input)))
labels=true_w[0]* features[:,0]+true_w[1]*features[:,1]+true_b[0]
labels += torch.from_numpy(np.random.normal(0,0.1,num_samples))

In [8]:
labels.shape

torch.Size([1000])

In [10]:
def data_iter(features,labels,batch_sizes):
    len_s=len(features)
    indices=list(range(len_s))
    np.random.shuffle(indices)
    for i in range(0,len_s,batch_sizes):
        j=torch.LongTensor(indices[i:min(i+batch_sizes,len_s)])
        yield features.index_select(0,j),labels.index_select(0,j)

In [12]:
# for i,j in data_iter(features,labels,20):
#     print(i,j)
#     break

In [13]:
def lin_reg(w,b,x):
    return torch.mm(x,w)+b

In [14]:
def l2_loss(y_hat,y):
    return (y_hat-y.view(y_hat.size()))**2/2

In [16]:
def batch_sgd(params,lm,x):
    for param in params:
        param.data -= lm*param.grad/len(x)
    

In [27]:
w=torch.tensor(np.random.randn(2),dtype=torch.float64).view(2,1)
b=torch.tensor([1],dtype=torch.float64)
w.requires_grad_(True)
b.requires_grad_(True)
batch_sizes=50
epochs=3
lm=1
for i in range(epochs):
    for x,y in data_iter(features,labels,batch_sizes):
        l=l2_loss(lin_reg(w,b,x),y).sum()
        l.backward()
        batch_sgd([w,b],lm,x)
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l=l2_loss(lin_reg(w,b,features),labels).mean().item()
    print('epoch : %d loss : %f ' % (i,train_l))

epoch : 0 loss : 0.005343 
epoch : 1 loss : 0.005593 
epoch : 2 loss : 0.005656 


In [26]:
print(true_w,'train w ',w)
print(true_b,'train b ',b)

[4, -3.5, 2] train w  tensor([[ 3.9796],
        [-3.4508]], dtype=torch.float64, requires_grad=True)
[6.1] train b  tensor([6.1017], dtype=torch.float64, requires_grad=True)
